Practical data scince course- Classification project
July 2021
Ido Shirat; 201116175

# Goal

I was hired by "cod-cod" company.
My goal is to gain data about boardgames and predict boardgame popularity and share it with the board.

# Scoring

Board expcet me to reach a minimum of 95% accuracy

# Import packages

In [1]:
# #<! Installing updated version of XGBoost
# !pip uninstall --yes xgboost
# !pip install xgboost

In [2]:
# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# SK Learn
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler, StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score


# XGBoost
import xgboost as xgb

# Trees Visualization
import pydot
from sklearn.tree import export_graphviz
# from sklearn.externals.six import StringIO

# Misc
import warnings
from sys import modules
from time import time
from platform import python_version

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# IPython
from IPython.display import Image, display

# Confuguration
warnings.filterwarnings('ignore')
%matplotlib inline

# **EDA**

**Data inspection**

In [3]:
df_raw = pd.read_csv('train.csv')
SHOW_PLOTS_AND_TABELS= False

if SHOW_PLOTS_AND_TABELS:
    print(df_raw.sample(5))
    print(df_raw.info())



In [4]:
if SHOW_PLOTS_AND_TABELS:
    sns.pairplot(df_train_raw)

In [5]:
if SHOW_PLOTS_AND_TABELS:
    plt.style.use('ggplot')
    plt.figure(figsize= (12,8))
    sns.histplot(df_raw.geek_rating, bins=50,color='r')

In [6]:
## create a multiclass descrete rating
df = df_raw

# df['Boardgame_rating'] = np.nan
df.loc[df.geek_rating<6,'Boardgame_rating'] = 1
df.loc[(df.geek_rating<6.5)&(df.geek_rating>=6),'Boardgame_rating'] = 2
df.loc[df.geek_rating>=6.5,'Boardgame_rating'] = 3
df['Boardgame_rating'].value_counts()
df['Boardgame_rating'] = df['Boardgame_rating'].astype(str)
df = df.drop(columns= ['geek_rating','game_id'])

for c in df.columns[df.dtypes=='object']:
    df[c] = df[c].str.lower()
    df[c] = df[c].str.replace(' ','')
#     df[c] = df[c].str.replace(',','')
    df[c] = df[c].str.replace('<','')
    df[c] = df[c].str.replace('>','')
    df[c] = df[c].str.replace('','')
    df[c] = df[c].str.replace(']','')
    df[c] = df[c].str.replace('[','')

In [7]:
## corr to rating
if SHOW_PLOTS_AND_TABELS:
    plt.style.use('ggplot')

    for c in df.columns[df.dtypes== np.int64]:
        plt.figure()
        sns.jointplot( x= df[c], y=df['Boardgame_rating'])


In [8]:
# Duplications
df.names.value_counts()
df[df.names =='cosmic encounter']

,names,min_players,max_players,avg_time,min_time,max_time,year,num_votes,age,mechanic,...,category5,category6,category7,category8,category9,category10,category11,category12,designer,Boardgame_rating


In [9]:
# a = df[['names','Boardgame_rating']]
# a.names = a.names.str.lower()
# a.names = a.names.str.split(' ')
# b = a.explode('names')
# b.Boardgame_rating = b.Boardgame_rating.astype(float)
# pop = b.names.value_counts()[b.names.value_counts()>30].index
# c = b.groupby('names').Boardgame_rating.mean()
# c.loc[pop]

# df.Boardgame_rating = df.Boardgame_rating.astype(float)
# df.loc[df.category1.str.contains('Building'),'Boardgame_rating'].mean()


In [10]:
# df[df['designer_0'].isin(df['designer_0'].value_counts()[df['designer_0'].value_counts()>20].index)].groupby('designer_0').Boardgame_rating.mean()

In [11]:
## nans
if SHOW_PLOTS_AND_TABELS:
    df.info()

**Data splitting**

In [12]:
## split designer
def desinger_split_func(df):
    df['designer_split'] = pd.DataFrame(data=df.designer.str.split(','))
    designer_split_df = pd.DataFrame(df.designer_split.tolist())
    for col in designer_split_df.columns:
        designer_split_df = designer_split_df.rename(columns={col:f'designer_{col}'})
    df = pd.concat([df,designer_split_df],axis=1)
    df = df.drop(columns=['designer','designer_split'])
    return df
df = desinger_split_func(df)



In [13]:
# split mechanic 
def mechanic_split_func(df):
    df['mechanic_split'] = pd.DataFrame(data=df.mechanic.str.split(','))
    mechanic_split_df = pd.DataFrame(df.mechanic_split.tolist())
    for col in mechanic_split_df.columns:
        mechanic_split_df = mechanic_split_df.rename(columns={col:f'mechanic_{col}'})
    df = pd.concat([df,mechanic_split_df],axis=1)
    df = df.drop(columns=['mechanic','mechanic_split'])
    return df

df = mechanic_split_func(df)

In [14]:
# df.Boardgame_rating = df.Boardgame_rating.astype(float)
# sns.pairplot(df)
# df.Boardgame_rating = df.Boardgame_rating.astype(str)
# df.Boardgame_rating = df.Boardgame_rating.astype(str)
df[df.columns[df.columns.str.contains('designer')]].sample(5)

,designer_0,designer_1,designer_2,designer_3,designer_4,designer_5,designer_6,designer_7,designer_8,designer_9,designer_10
1553,reinerknizia,None,None,None,None,None,None,None,None,None,None
671,brunocathala,sergelaget,None,None,None,None,None,None,None,None,None
2297,stevejackson(i),None,None,None,None,None,None,None,None,None,None
2538,chriszinsli,suzannezinsli,None,None,None,None,None,None,None,None,None
474,alessandrozucchini,None,None,None,None,None,None,None,None,None,None


**Data cleanning**

In [15]:
## geek_rating_cat
if SHOW_PLOTS_AND_TABELS:
    sns.histplot(df.Boardgame_rating, bins=50)


In [16]:
## min_players
if SHOW_PLOTS_AND_TABELS:
    plt.style.use('ggplot')
    plt.figure(figsize=(12,8))
    sns.jointplot(df.min_players, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.min_players, bins=50)
df.min_players = df.min_players.astype(str)

In [17]:
# ## max_players
if SHOW_PLOTS_AND_TABELS:
    plt.style.use('ggplot')
    plt.figure(figsize=(12,8))
    sns.jointplot(df.max_players, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.max_players, bins=50)

df = df[df['max_players']<50]
df.max_players = df.max_players.astype(str)

if SHOW_PLOTS_AND_TABELS:
    plt.figure(figsize=(12,8))
    sns.jointplot(df.max_players, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.max_players, bins=50)

In [18]:
## avg_time
if SHOW_PLOTS_AND_TABELS:
    plt.style.use('ggplot')
    plt.figure(figsize=(12,8))
    sns.jointplot(df.avg_time, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.avg_time, bins=50)


df = df[(df['avg_time']<2000)&(df['avg_time']>0)]

if SHOW_PLOTS_AND_TABELS:
    plt.figure(figsize=(12,8))
    sns.jointplot(df.avg_time, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.avg_time, bins=50)

In [19]:
## min_time
if SHOW_PLOTS_AND_TABELS:
    plt.style.use('ggplot')
    plt.figure(figsize=(12,8))
    sns.jointplot(df.min_time, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.min_time, bins=50)
df = df[df['min_time']>0]



In [20]:
## max_time
if SHOW_PLOTS_AND_TABELS:
    plt.style.use('ggplot')
    plt.figure(figsize=(12,8))
    sns.jointplot(df.max_time, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.max_time, bins=50)
df = df[df['max_time']>0]


In [21]:
## year
if SHOW_PLOTS_AND_TABELS:
    plt.style.use('ggplot')
    plt.figure(figsize=(12,8))
    sns.jointplot(df.year, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.year, bins=50)

df = df[df.year>1000]

if SHOW_PLOTS_AND_TABELS:
    plt.figure(figsize=(12,8))
    sns.jointplot(df.year, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.year, bins=50)

In [22]:
## num_votes
if SHOW_PLOTS_AND_TABELS:
    plt.style.use('ggplot')
    plt.figure(figsize=(12,8))
    sns.jointplot(df.num_votes, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.num_votes, bins=50)

df = df[(df.num_votes>(df.num_votes.mean()-df.num_votes.std()*3))&(df.num_votes<(df.num_votes.mean()+df.num_votes.std()*3))]

if SHOW_PLOTS_AND_TABELS:
    plt.figure(figsize=(12,8))
    sns.jointplot(df.num_votes, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.num_votes, bins=50)

In [23]:
## age
if SHOW_PLOTS_AND_TABELS:
    plt.style.use('ggplot')
    plt.figure(figsize=(12,8))
    sns.jointplot(df.age, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.age, bins=50)

df = df[df.age<30]

if SHOW_PLOTS_AND_TABELS:
    plt.figure(figsize=(12,8))
    sns.jointplot(df.age, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.age, bins=50)

In [24]:
## owned
if SHOW_PLOTS_AND_TABELS:
    plt.style.use('ggplot')
    plt.figure(figsize=(12,8))
    sns.jointplot(df.owned, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.owned, bins=50)

df = df[(df.owned>(df.owned.mean()-df.owned.std()*3))&(df.owned<(df.owned.mean()+df.owned.std()*3))]

if SHOW_PLOTS_AND_TABELS:
    plt.figure(figsize=(12,8))
    sns.jointplot(df.owned, df.Boardgame_rating)
    plt.figure(figsize=(12,8))
    sns.histplot(df.owned, bins=50)

In [25]:
## designer

def rating_groups_func(df, col_type):
    df_groups_rating = pd.DataFrame()
    for col in df[df.columns[df.columns.str.contains(col_type)]]:
        a = df[[col,'Boardgame_rating']].rename(columns= {col:col_type})
        df_groups_rating = pd.concat([df_groups_rating, a],axis=0)
    df_groups_rating = df_groups_rating[~df_groups_rating.isna()]
    
    df_groups_rating = df_groups_rating.groupby(col_type,as_index=False).Boardgame_rating.mean()
    df_groups_rating[f'{col_type}_group'] = np.nan
    for inx,row in df_groups_rating.iterrows():
        df_groups_rating.loc[inx,f'{col_type}_group'] = df.Boardgame_rating.unique()[np.argmin(abs(row.Boardgame_rating-df.Boardgame_rating.unique()))]

    for col in df[df.columns[df.columns.str.contains(col_type)]]:
        col_df= df_groups_rating.rename(columns={col_type:col,f'{col_type}_group':f'{col}_group'})
        df = pd.merge(df,col_df[[col,f'{col}_group']], how='left', on=col)
    df[f'{col_type}_rating_mean'] = df[df.columns[df.columns.str.contains('group')]].mean(axis=1)
    
    if SHOW_PLOTS_AND_TABELS:
        plt.figure()
        sns.scatterplot(x=df[f'{col}_group'],y=df.Boardgame_rating)
    return df

def object_type_agg_dummies(df,col_type):
    a= df[df.columns[df.columns.str.contains(col_type)]]
    b=a.stack().str.get_dummies().sum(level=0)
    df = pd.concat([df.reset_index(),b.reset_index()],axis=1)
    return df

df = object_type_agg_dummies(df,'designer')

df.Boardgame_rating = df.Boardgame_rating.astype(float)
df = rating_groups_func(df, 'designer')
df.Boardgame_rating = df.Boardgame_rating.astype(str)


df = df.drop(columns=df[df.columns[(df.columns.str.contains('designer'))&(df.dtypes==object)]].columns)



In [26]:
## category

df = object_type_agg_dummies(df,'category')

df.Boardgame_rating = df.Boardgame_rating.astype(float)
df = rating_groups_func(df, 'category')
df.Boardgame_rating = df.Boardgame_rating.astype(str)

df = df.drop(columns=df[df.columns[(df.columns.str.contains('category'))&(df.dtypes==object)]].columns)


In [27]:
## mechanic
df= df.drop(columns=['index','level_0'])
df = object_type_agg_dummies(df,'mechanic')

df.Boardgame_rating = df.Boardgame_rating.astype(float)
df = rating_groups_func(df, 'mechanic')
df.Boardgame_rating = df.Boardgame_rating.astype(str)

df = df.drop(columns=df[df.columns[(df.columns.str.contains('mechanic'))&(df.dtypes==object)]].columns)
df= df.drop(columns='index')


**Feature enegeniring**

In [28]:
df['num_owner_per_year'] = df.owned/ (2021 - df.year)
df['num_votes_per_year'] = df.num_votes/ (2021 - df.year)
df['owned_to_votes_ratio'] = df.owned/df.num_votes
df['num_votes_log'] = np.log10(df['num_votes'])



# if SHOW_PLOTS_AND_TABELS:
#     sns.scatterplot(x=df['owned_to_votes_ratio'],y=df.Boardgame_rating)
for c in df.columns:
    df[c] = df[c].fillna(0)
df=df[df.columns[~df.columns.duplicated(keep=False)]]

In [29]:
## predictro set to dummies; y= predicted values
X = pd.get_dummies(df.drop(columns= 'Boardgame_rating'))
y = df['Boardgame_rating']

In [30]:
## X and y split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 1234,test_size= 0.3 ,stratify= df['Boardgame_rating'])


In [59]:
## train set scaling
##MinMaxScaler,MaxAbsScaler, StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(data= X_train_scaled,columns=X_train.columns)


3125    2.0
370     1.0
2032    1.0
776     3.0
1503    3.0
       ... 
2375    1.0
2954    1.0
1614    3.0
1795    3.0
1985    1.0
Name: Boardgame_rating, Length: 2241, dtype: object

In [32]:
## select best feautres
best_features_f = SelectKBest(f_classif, k=2300).fit(X_train_scaled, y_train)
best_features_m = SelectKBest(mutual_info_classif, k=2300).fit(X_train_scaled, y_train)

X_train_scaled_best_features = X_train_scaled[X_train_scaled.columns[best_features_f.get_support()*best_features_m.get_support()]]
# X_train_scaled_best_features = X_train_scaled[X_train_scaled.columns[best_features_f.get_support()]]


# **Modeling data**

Gradient boosting

In [105]:
## Grid search
numSplits = 4 
paramGrid = {'learning_rate': [0.1,0.15,0.2], 'n_estimators': [100,130,150,180], 'max_depth': [3, 5, 7]}
# paramGrid = {'n_estimators': [80, 100, 120]}

## KNeighborsClassifier; LogisticRegression; DecisionTreeClassifier

# gsModel = GridSearchCV(estimator = xgb.XGBClassifier(verbosity = 0, random_state = 1234,class_weight= 'balanced'), param_grid = paramGrid, scoring = 'accuracy', cv = numSplits) #<! ~ 210 [Sec]
# gsModel = xgb.XGBClassifier(verbosity = 0, random_state = 1234, class_weight = 'balanced',learning_rate= 0.1, max_depth= 3, n_estimators= 300) #<! ~ 210 [Sec]
# gsModel = BaggingClassifier(base_estimator= KNeighborsClassifier(n_neighbors=3),n_estimators=150) #<! ~ 210 [Sec]
gsModel = AdaBoostClassifier(base_estimator= LogisticRegression(class_weight='balanced',random_state = 1234,C=1),n_estimators=150,learning_rate=0.1) #<! ~ 210 [Sec]


t1 = time()
fit = gsModel.fit(X_train_scaled_best_features, y_train)
t2 = time()
print(f"XGBoost took {t2-t1:.2f} seconds")



XGBoost took 15.48 seconds


In [ ]:
## print best HP
if type(gsModel)==GridSearchCV:
    print(gsModel.best_params_)

## updated Best params
## n_log_loss{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 150}
## balanced_accuracy{'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 180}
## accuracy{'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 130}

In [ ]:
## when running to mom grid search, get feature importance
if type(gsModel)!=GridSearchCV:
    xgb_fea_imp=pd.DataFrame(list(fit.get_booster().get_fscore().items()),
    columns=['feature','importance']).sort_values('importance', ascending=False)
    xgb_fea_imp[xgb_fea_imp.importance>20].set_index('feature').plot.bar(figsize=(12,8))

In [106]:
##print scores
def report(clf, X, y):
    cm = pd.DataFrame(confusion_matrix(y_true = y, y_pred = clf.predict(X)), 
                      index = clf.classes_, 
                      columns = clf.classes_)
    rep = classification_report(y_true = y, y_pred = clf.predict(X))
    print('confusion matrix:\n',cm,'\n')
    print('classification report:\n',rep)
    print('log_loss',log_loss(y_pred= clf.predict_proba(X),y_true= y))
    
report(gsModel, X_train_scaled_best_features, y_train)


confusion matrix:
       1.0  2.0  3.0
1.0  1225  124    2
2.0    46  507   20
3.0     0   16  301 

classification report:
               precision    recall  f1-score   support

         1.0       0.96      0.91      0.93      1351
         2.0       0.78      0.88      0.83       573
         3.0       0.93      0.95      0.94       317

    accuracy                           0.91      2241
   macro avg       0.89      0.91      0.90      2241
weighted avg       0.91      0.91      0.91      2241

log_loss 0.7327412275373023


In [107]:
## Validation of the model using the test set
X_test_scaled = scaler.fit_transform(X_test)
X_test_scaled = pd.DataFrame(data= X_test_scaled,columns=X_test.columns)
X_test_scaled_best_features = X_test_scaled[X_test_scaled.columns[X_test_scaled.columns.isin(X_train_scaled_best_features.columns)]]

report(gsModel, X_test_scaled_best_features, y_test)


confusion matrix:
      1.0  2.0  3.0
1.0  504   73    2
2.0   33  193   20
3.0    1   41   94 

classification report:
               precision    recall  f1-score   support

         1.0       0.94      0.87      0.90       579
         2.0       0.63      0.78      0.70       246
         3.0       0.81      0.69      0.75       136

    accuracy                           0.82       961
   macro avg       0.79      0.78      0.78       961
weighted avg       0.84      0.82      0.83       961

log_loss 0.7985856234276654


Random forest

In [ ]:
## grid search
numSplits = 4 #<! What's the equivalent Train / Test split?
paramGrid = {'learning_rate': [0.15,0.2,0.25], 'n_estimators': [70,100,130,150], 'max_depth': [3, 5, 7,9,12]}
# paramGrid = {'n_estimators': [80, 100, 120]}

# gsModel = GridSearchCV(estimator = xgb.XGBRFClassifier(verbosity = 0, random_state = 1234,class_weight= 'balanced'), param_grid = paramGrid, scoring = 'neg_log_loss', cv = numSplits) #<! ~ 210 [Sec]
gsModel = xgb.XGBRFClassifier(verbosity = 0, random_state = 1234,class_weight= 'balanced',n_estimator= 100,learning_rate= 0.25,max_depth= 12) #<! ~ 210 [Sec]


t1 = time()
fit = gsModel.fit(X_train_scaled_best_features, y_train)
t2 = time()
print(f"XGBoost took {t2-t1:.2f} seconds")


In [ ]:
## best params
if type(gsModel)==GridSearchCV:
    print(gsModel.best_params_)

## updated Best params
## n_log_loss{'learning_rate': 0.25, 'max_depth': 12, 'n_estimators': 100}


In [ ]:
## non GS feature importance
if type(gsModel)!=GridSearchCV:
    xgb_fea_imp=pd.DataFrame(list(fit.get_booster().get_fscore().items()),
    columns=['feature','importance']).sort_values('importance', ascending=False)
    xgb_fea_imp[xgb_fea_imp.importance>20].set_index('feature').plot.bar(figsize=(12,8))

In [ ]:
##train set score
report(gsModel, X_train_scaled_best_features, y_train)


In [ ]:
##test set validation
X_test_scaled = scaler.fit_transform(X_test)
X_test_scaled = pd.DataFrame(data= X_test_scaled,columns=X_test.columns)
X_test_scaled_best_features = X_test_scaled[X_test_scaled.columns[X_test_scaled.columns.isin(X_train_scaled_best_features.columns)]]

report(gsModel, X_test_scaled_best_features, y_test)